In [24]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.
import torch.nn as nn
from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import wandb

In [25]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [26]:
class CustomDataset(Dataset):
    def __init__(self, df, tokenizer, input_len, summ_len, is_train=True):
        self.tokenizer = tokenizer
        self.df = df
        self.source_len = input_len
        self.summ_len = summ_len
        self.is_train = is_train
        if self.is_train:
            self.input_ids = tokenizer(self.df['dialogue'].tolist(), return_tensors="pt", padding=True,
                                add_special_tokens=True, truncation=True, max_length=512, return_token_type_ids=False).input_ids
            self.labels = tokenizer(self.df['summary'].tolist(), return_tensors="pt", padding=True,
                                add_special_tokens=True, truncation=True, max_length=100, return_token_type_ids=False).input_ids
        else:
            self.input_ids = tokenizer(self.df['dialogue'].tolist(), return_tensors="pt", padding=True,
                                add_special_tokens=True, truncation=True, max_length=512, return_token_type_ids=False).input_ids
    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        if self.is_train:
            return self.input_ids[idx], self.labels[idx]
        else:
            return self.input_ids[idx]

In [27]:
tokenizer = AutoTokenizer.from_pretrained("psyche/KoT5-summarization")

In [28]:
special_tokens_dict={'additional_special_tokens': ['#Person1#', '#Person2#','#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#PhoneNumber#', 
                                                   '#Address#', '#PassportNumber#', '#CardNumber#', '#Email#', '#DateOfBirth#',]}

tokenizer.add_special_tokens(special_tokens_dict)

13

In [29]:
train_df = pd.read_csv('../data/new_train.csv')
val_df = pd.read_csv('../data/new_dev.csv')

In [30]:
train_dataset = CustomDataset(train_df[['dialogue', 'summary']], tokenizer, 400, 256)
val_dataset = CustomDataset(val_df[['dialogue', 'summary']], tokenizer, 400, 256)

In [35]:
train_params = {
    'batch_size': 8,
    'shuffle': True,
    'num_workers': 0
}

val_params = {
    'batch_size': 8,
    'shuffle': False,
    'num_workers': 0
}

train_loader = DataLoader(train_dataset, **train_params)
val_loader = DataLoader(val_dataset, **val_params)

In [36]:
def ids_to_words(tokenizer, preds, labels):
    decoded_preds = tokenizer.batch_decode(preds, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 정확한 평가를 위해 미리 정의된 불필요한 생성토큰들을 제거합니다.
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = ['<usr>', f"{tokenizer.unk_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]
    return replaced_predictions, replaced_labels

In [37]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
def compute_metrics(replaced_predictions, replaced_labels):
    rouge = Rouge()

    # 최종적인 ROUGE 점수를 계산합니다.
    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)

    # ROUGE 점수 중 F-1 score를 통해 평가합니다.
    result = {key: value["f"] for key, value in results.items()}
    return result

In [38]:
def train(epoch, model, device, train_loader, optimizer, log_interval, train_step):
    model.train()
    current_loss = 0.0
    for batch_idx, data in enumerate(train_loader):

        input_ids = data[0].to(device, dtype=torch.long)
        labels = data[1].to(device, dtype=torch.long)

        loss = model(input_ids=input_ids, labels=labels).loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        current_loss += loss.item()
        if (batch_idx + 1) % log_interval == 0:
            train_loss = current_loss / batch_idx

            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, 
                batch_idx * len(input_ids), 
                len(train_loader) * len(input_ids), 
                100 * batch_idx / len(train_loader), 
                train_loss))
            
            if wandb is not None:
                wandb.log({
                    "Loss/Train": train_loss, 
                }, step=train_step)
        train_step += 1
    return train_step

In [39]:
def validate(tokenizer, model, device, val_loader, train_step):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for val_idx, data in enumerate(val_loader):
            input_ids = data[0].to(device, dtype=torch.long)
            labels = data[1].to(device, dtype=torch.long)

            pred_ids = model.generate(
                input_ids=input_ids,
                max_length=256, 
                num_beams=4,
                repetition_penalty=2.0, 
                length_penalty=1.0, 
                early_stopping=True,
                no_repeat_ngram_size=2
            )

            loss = model(input_ids=input_ids, labels=labels).loss
            total_loss += loss.item()
            replaced_predictions, replaced_labels = ids_to_words(tokenizer, pred_ids, labels)
            result = compute_metrics(replaced_predictions, replaced_labels)

        val_loss = total_loss / len(val_loader)
        if wandb is not None:
            wandb.log({
                "Loss/Val": total_loss,
                "Rouge-1/Val": result['rouge-1'],
                "Rouge-2/Val": result['rouge-2'],
                "Rouge-L/Val": result['rouge-l'],
            }, step=train_step)
        print("Validation: Val Loss: {:.6f}, Rouge-1/Val: {:.6f}, Rouge-2/Val: {:.6f}, Rouge-l/Val: {:.6f}".format(val_loss, result['rouge-1'], result['rouge-2'], result['rouge-l']))
        
        print('-'*150)
        print(f"PRED: {replaced_predictions[0]}")
        print(f"GOLD: {replaced_labels[0]}")
        print('-'*150)
        print(f"PRED: {replaced_predictions[1]}")
        print(f"GOLD: {replaced_labels[1]}")
        print('-'*150)
        print(f"PRED: {replaced_predictions[2]}")
        print(f"GOLD: {replaced_labels[2]}")
        print()
    

In [40]:
def predict(tokenizer, model, device, test_loader, fname):
    model.eval()
    summary = []
    with torch.no_grad():
        for input_ids in tqdm(test_loader):
            input_ids = input_ids.to(device, dtype=torch.long)

            pred_ids = model.generate(
                input_ids=input_ids,
                max_length=256, 
                num_beams=4,
                repetition_penalty=2.0, 
                length_penalty=1.0, 
                early_stopping=True,
                no_repeat_ngram_size=2
            )
            for ids in pred_ids:
                result = tokenizer.decode(ids)
                summary.append(result)
                
    remove_tokens = ['<usr>', f"{tokenizer.unk_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    preprocessed_summary = summary.copy()
    for token in remove_tokens:
        preprocessed_summary = [sentence.replace(token," ") for sentence in preprocessed_summary]

    output = pd.DataFrame(
        {
            "fname": fname,
            "summary" : preprocessed_summary,
        }
    )
    return output


In [41]:
epoch = 10

log_interval = 310
model = AutoModelForSeq2SeqLM.from_pretrained("psyche/KoT5-summarization").to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)

run_name = f"KoT5-summarization-fifth-test"
log_dir = f"runs/{run_name}"

project_name = 'DS'
run_tags=[project_name]
wandb.init(
    entity="ch_hee",
    project=project_name,
    name=run_name,
    tags=run_tags,
    reinit=True
)


Loss/Train,█▇▆▆▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁
Loss/Val,▄▁▁▁▁▃▄▅▆█
Rouge-1/Val,▃█▆▄▁▁▅▂▂▂
Rouge-2/Val,▄▅█▃▁▂▄▄▄▃
Rouge-L/Val,▄██▆▁▂▆▂▂▁
Loss/Train,0.24355
Loss/Val,39.44158
Rouge-1/Val,0.40597
Rouge-2/Val,0.15798
Rouge-L/Val,0.35636


In [42]:
log_dir = './logs'
model_path = os.path.join(log_dir, 'models-fifth')
os.makedirs(model_path, exist_ok=True)

In [43]:
save_point = 0
path = os.path.join(model_path, 'model.ckpt')

In [44]:
train_step = 0
for epoch in range(1, epoch + 1):
    train_step = train(epoch, model, device, train_loader, optimizer, log_interval, train_step)
    validate(tokenizer, model, device, val_loader, train_step)
    save_dir = os.path.dirname(path)
    torch.save(model, os.path.join(save_dir, f'epoch-{epoch}-{path.split("/")[-1]}'))
    print('Saved Model!!!')



Train Epoch: 1 [2472/12464 (20%)]	Train Loss: 0.904415
Train Epoch: 1 [4952/12464 (40%)]	Train Loss: 0.785810
Train Epoch: 1 [7432/12464 (60%)]	Train Loss: 0.745230
Train Epoch: 1 [9912/12464 (80%)]	Train Loss: 0.712366
Train Epoch: 1 [12392/12464 (99%)]	Train Loss: 0.690205
Validation: Val Loss: 0.554155, Rouge-1/Val: 0.380810, Rouge-2/Val: 0.143115, Rouge-l/Val: 0.380810
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:   차에서 이상한 소리가 나서 #Person2# 은 브레이크를 새로 교체해야 합니다. #Person1# 는 내일 아침에 차를 다시 가져오는 것을 제안하지만, 그것은 좋은 생각이 아닙니다.                  
GOLD: #Person2# 의 차에서 이상한 소리가 납니다. #Person1# 는 브레이크를 교체해야 할 것으로 생각하지만, #Person1# 는 내일까지 그것을 고칠 수 없습니다. #Person2# 는 오늘 밤에 공연을 보러 가고 싶어합니다; #Person1# 는 #Person2# 에게 버스를 이용할 것을 제안합니다.                                      
-----------------------------------------------------------------------------------------------------------------------------

In [19]:
test_df = pd.read_csv('../data/new_test.csv')

In [20]:
test_dataset = CustomDataset(test_df[['dialogue']], tokenizer, 512, 100, is_train=False)

In [21]:
test_params = {
    'batch_size': 8,
    'shuffle': False,
    'num_workers': 0
}

test_loader = DataLoader(test_dataset, **test_params)

In [22]:
best_model = torch.load('../code/logs/models-fourth/epoch-10-model.ckpt')

In [23]:
output = predict(tokenizer, model, device, test_loader, test_df['fname'])
output

  0%|          | 0/63 [00:00<?, ?it/s]

100%|██████████| 63/63 [01:06<00:00,  1.06s/it]


,fname,summary
0,test_0,더슨 씨는 #Person1# 에게 모든 사무실 통신은 이메일 통신과 공식 메모로...
1,test_1,#Person1# 은 또 교통 체증에 걸렸다. #Person2# 는 #Person...
2,test_2,케이트는 #Person1# 에게 마샤와 히어로가 별거 중이다가 이혼을 신청했다고...
3,test_3,#Person1# 은 브라이언의 생일을 축하하고 파티에 초대한다. 브라이언은 #P...
4,test_4,"#Person2# 는 #Person1# 에게 올림픽 공원의 크기, 좌석 수, 그리..."
...,...,...
494,test_495,잭이 찰리에게 집에 와서 비디오 게임을 하자고 제안한다. 찰리는 자신이 캐릭터를...
495,test_496,#Person2# 는 #Person1# 에게 어떻게 컨트리 음악에 관심을 가지게 ...
496,test_497,앨리스는 #Person1# 에게 세탁기와 건조기 안에 비누를 넣지 않았다고 말합...
497,test_498,매튜는 계약이 다음 달에 끝나기 때문에 집을 찾고 있다. 스티브는 분류된 광고를...
